In [1]:
import cv2 as cv
import numpy as np
import random
import os
import math
import xml.etree.ElementTree as ET

In [2]:
def click_corners(img, objpoints, imgpoints, objp, window, n_rows=8, n_cols=6):
    """
    get 4 corners from user clicks (in order) and use linear interpolation to get the
    other inner points.
    """

    order = ["TL", "BL", "TR", "BR"]
    box_corners = []

    # function to handle clicks on the image
    def click_event(event, x, y, flags, param):

        box_corners, img, window = param

        # check for left mouse clicks
        if event == cv.EVENT_LBUTTONDOWN and len(box_corners) < 4:

            box_corners.append((x, y))

            # display clicked points
            # on the image window
            font = cv.FONT_HERSHEY_SIMPLEX
            cv.putText(
                img, order[len(box_corners) - 1], (x, y), font, 1, (255, 0, 0), 2
            )
            cv.imshow(window, img)

    cv.namedWindow(window, cv.WINDOW_NORMAL)
    cv.imshow(window, img)
    cv.setMouseCallback(window, click_event, [box_corners, img, window])
    # wait for a key to be pressed to exit
    # print("click on the 4 corners, then press any key.")
    cv.waitKey(0)

    tl, bl, tr, br = np.array(box_corners, dtype=np.float32)

    # get first and last columns of points with linear interpolation
    first_col = np.linspace(tl, bl, n_rows)
    last_col = np.linspace(tr, br, n_rows)

    # get rest corner points by linearly interpolating the two columns
    all_points = np.vstack(
        [np.linspace(first_col[i], last_col[i], n_cols) for i in range(n_rows)]
    )
    corners = all_points.reshape(-1, 1, 2)

    objpoints.append(objp)
    imgpoints.append(corners)

    cv.destroyWindow(window)
    cv.drawChessboardCorners(img, (n_rows, n_cols), corners, True)
    cv.imshow("Interpolated Chessboard Corners", img)
    cv.waitKey(0)
    cv.destroyAllWindows()


def find_auto(img, gray, objpoints, imgpoints, objp, window, n_rows=9, n_cols=6):
    """
    used to find the corner points and fill objpoints, imgpoints lists
    img, gray: original and grayscale image
    """

    # choice task - denoising
    gray = cv.fastNlMeansDenoising(gray, None, h=10)

    # detect chess board corners
    ret, corners = cv.findChessboardCorners(gray, (n_rows, n_cols), None)

    if ret:
        # improve quality of automatically found corners
        criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
        corners = cv.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)

        objpoints.append(objp)
        imgpoints.append(corners)

        # display the corners on image
        # cv.drawChessboardCorners(img, (n_rows, n_cols), corners, True)
        # cv.imshow(window, img)
        # cv.waitKey(0)
        # cv.destroyWindow(window)
    # else:
    #     # if corners are not found, switch to manual mode
    #     click_corners(img, objpoints, imgpoints, objp, window, n_rows, n_cols)


def flat_str(matrix, vec=False):

    if vec:
        flat = [str(element) for element in matrix.flatten().tolist()]
    else:
        flat = ["  ".join(str(element) for element in row) for row in matrix.tolist()]

    return "\n  " + "\n  ".join(flat) + "\n"


def save_config(mtx, dist, R, tvec, video=1):

    opencv_storage = ET.Element("opencv_storage")
    camera_matrix = ET.SubElement(
        opencv_storage, "CameraMatrix", type_id="opencv-matrix"
    )
    ET.SubElement(camera_matrix, "rows").text = "3"
    ET.SubElement(camera_matrix, "cols").text = "3"
    ET.SubElement(camera_matrix, "dt").text = "f"
    ET.SubElement(camera_matrix, "data").text = flat_str(mtx)

    distortion_coeffs_elem = ET.SubElement(
        opencv_storage, "DistortionCoeffs", type_id="opencv-matrix"
    )
    ET.SubElement(distortion_coeffs_elem, "rows").text = "5"
    ET.SubElement(distortion_coeffs_elem, "cols").text = "1"
    ET.SubElement(distortion_coeffs_elem, "dt").text = "f"
    ET.SubElement(distortion_coeffs_elem, "data").text = flat_str(dist, True)

    rotation_matrix_elem = ET.SubElement(
        opencv_storage, "RotationMatrix", type_id="opencv-matrix"
    )
    ET.SubElement(rotation_matrix_elem, "rows").text = "3"
    ET.SubElement(rotation_matrix_elem, "cols").text = "3"
    ET.SubElement(rotation_matrix_elem, "dt").text = "f"
    ET.SubElement(rotation_matrix_elem, "data").text = flat_str(R)

    translation_vector_elem = ET.SubElement(
        opencv_storage, "TranslationVector", type_id="opencv-matrix"
    )
    ET.SubElement(translation_vector_elem, "rows").text = "3"
    ET.SubElement(translation_vector_elem, "cols").text = "1"
    ET.SubElement(translation_vector_elem, "dt").text = "f"
    ET.SubElement(translation_vector_elem, "data").text = flat_str(tvec, True)

    tree = ET.ElementTree(opencv_storage)
    # write to file
    with open(f"data/cam{video}/config.xml", "wb") as file:
        tree.write(file, encoding="utf-8", xml_declaration=True)


def get_checkboard(video=1):

    path = f"./data/cam{video}/checkerboard.avi"
    cam = cv.VideoCapture(path)

    j = 0
    frames = []
    while j <= 100:
        ret, frame = cam.read()
        if ret:
            j += 1
            frame = frame.astype(np.float32)
            frames.append(frame)
        else:
            break
    mean_frame = np.mean(frames, axis=0).astype(np.uint8)
    # Save the mean image
    out_path = f"data/cam{video}/checkboard_axes.jpg"
    cv.imwrite(out_path, mean_frame)
    print(f"Saved {out_path}")

    cam.release()


def get_calib_images(num=30, n_rows=8, n_cols=6, video=1):

    path = f"./data/cam{video}/intrinsics.avi"
    cam = cv.VideoCapture(path)

    if not cam.isOpened():
        print("Error: Could not open video file.")

    j = 0
    saved = 0
    while True:

        ret, frame = cam.read()
        if ret:
            j += 1
            gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
            gray = cv.fastNlMeansDenoising(gray, None, h=10)

            # detect chess board corners
            found, corners = cv.findChessboardCorners(gray, (n_rows, n_cols), None)
            if found and j % 10 == 0:
                saved += 1
                out_path = f"data/cam{video}/image{saved}.jpg"
                cv.imwrite(out_path, frame)
                print(f"Saved {out_path}")

            if saved >= num:
                break
        else:
            print("failed to grab frame")
            break

    cam.release()

In [ ]:
def detect_inner_chessboard_corners(
    img,
    video=1,
    n_rows=8,
    n_cols=6,
):
    """
    Detects the inner corners of a chessboard by:
    1. Extracting the background from a video.
    2. Subtracting the chessboard image from the background.
    3. Detecting the four outer chessboard corners.
    4. Computing the inner chessboard corners.

    :return: List of detected inner chessboard corners.
    """
    output_background_path = f"data/cam{video}/backgorund.jpg"
    chessboard_path = f"data/cam{video}/checkboard_axes.jpg"

    def extract_background(video=1, num_frames=100):
        """Extracts the background by computing the median frame over multiple frames."""
        cap = cv.VideoCapture(f"data/cam{video}/background.avi")

        frames = []
        frame_count = 0

        while frame_count <= num_frames:
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
            frames.append(frame)
            frame_count += 1

        cap.release()

        background = np.median(np.stack(frames, axis=0), axis=0).astype(np.uint8)
        return background

    def detect_chessboard_corners(background_img, chessboard_img):
        """Detects outer chessboard corners using improved edge detection."""
        gray_bg = background_img
        gray_chessboard = cv.cvtColor(chessboard_img, cv.COLOR_BGR2GRAY)

        # Step 1: Apply background subtraction
        diff = cv.absdiff(gray_bg, gray_chessboard)

        # Step 2: Preprocess image for better corner detection
        blurred = cv.GaussianBlur(diff, (5, 5), 0)
        edges = cv.Canny(blurred, 50, 150)  # Edge detection

        # Step 3: Find contours
        contours, _ = cv.findContours(edges, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

        best_quad = None
        max_area = 0

        for contour in contours:
            area = cv.contourArea(contour)
            if area > 5000:  # Ignore small contours
                epsilon = 0.02 * cv.arcLength(contour, True)
                approx = cv.approxPolyDP(contour, epsilon, True)

                if len(approx) == 4:  # Must be a quadrilateral
                    if area > max_area:
                        max_area = area
                        best_quad = approx

        if best_quad is None:
            print("Error: Could not detect chessboard corners.")
            return None

        # Step 4: Sort outer corners in order (TL, TR, BL, BR)
        best_quad = best_quad.reshape(4, 2)
        best_quad = sorted(best_quad, key=lambda x: x[1])  # Sort by y (top to bottom)
        top_corners = sorted(best_quad[:2], key=lambda x: x[0])  # Left to right
        bottom_corners = sorted(best_quad[2:], key=lambda x: x[0])  # Left to right

        ordered_corners = np.array(
            [top_corners[0], top_corners[1], bottom_corners[0], bottom_corners[1]],
            dtype=np.float32,
        )

        return ordered_corners

    def compute_internal_corners(outer_corners, rows=8, cols=6):
        """Computes internal chessboard corners using bilinear interpolation."""
        tl, tr, bl, br = outer_corners

        grid_x, grid_y = np.meshgrid(
            np.linspace(0, 1, cols + 2)[1:-1], np.linspace(0, 1, rows + 2)[1:-1]
        )

        internal_corners = []
        for i in range(rows):
            for j in range(cols):
                alpha, beta = grid_x[i, j], grid_y[i, j]
                point = (
                    (1 - alpha) * (1 - beta) * tl
                    + alpha * (1 - beta) * tr
                    + (1 - alpha) * beta * bl
                    + alpha * beta * br
                )
                internal_corners.append(point)

        return np.array(internal_corners, dtype=np.float32)

    # Step 1: Extract the background
    background_img = extract_background(video=1)
    cv.imwrite(output_background_path, background_img)

    # Step 2: Load chessboard image
    chessboard_img = cv.imread(chessboard_path)

    # Step 3: Detect outer chessboard corners
    outer_corners = detect_chessboard_corners(background_img, chessboard_img)

    # Step 4: Compute internal chessboard corners
    corners = compute_internal_corners(outer_corners, n_rows, n_cols)

    # Step 5: Draw detected corners on the chessboard image
    for point in corners:
        cv.circle(
            chessboard_img, tuple(point.astype(int)), 3, (255, 0, 0), -1
        )  # Blue inner corners
    for point in outer_corners:
        cv.circle(
            chessboard_img, tuple(point.astype(int)), 4, (0, 0, 255), -1
        )  # Red outer corners

    cv.drawChessboardCorners(img, (n_rows, n_cols), corners, True)
    cv.imshow("Interpolated Chessboard Corners", img)
    cv.waitKey(0)
    cv.destroyAllWindows()

    return corners


# inner_corners = detect_inner_chessboard_corners(objpoints, imgpoints, video=1)

# if inner_corners is not None:
#     print("Detected Inner Chessboard Corners (x, y):")
#     for i, pt in enumerate(inner_corners):
#         print(f"Corner {i+1}: {pt}")

## Intrinsics


In [14]:
def calibrate(video=1, n_rows=8, n_cols=6):

    objp = np.zeros((n_rows * n_cols, 3), np.float32)
    objp[:, :2] = np.mgrid[0:n_rows, 0:n_cols].T.reshape(-1, 2)
    objp = objp * 115

    # Lists to store object and image points from all images.

    objpoints = []  # 3d point in real world space

    imgpoints = []  # 2d points in image plane.

    for file in os.listdir(f"data/cam{video}"):
        if file.startswith("image"):

            img = cv.imread(f"data/cam{video}/{file}")
            gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

            # we convert to grayscale before passing to 'findChessboardCorners'
            window = file
            # find chess board corners either automatically or manually
            find_auto(
                img,
                gray,
                objpoints,
                imgpoints,
                objp,
                window,
                n_rows=n_rows,
                n_cols=n_cols,
            )

    ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(
        objpoints, imgpoints, gray.shape[::-1], None, None
    )

    return ret, mtx, dist, rvecs, tvecs

In [15]:
# get_calib_images(video=1)
# get_calib_images(video=2)
# get_calib_images(video=4)
# get_calib_images(video=3)

## Extrinsics


In [ ]:
def camera_params(mtx, dist, video=1, n_rows=8, n_cols=6, auto=False):

    # for extrinsics use checkerboard.avi
    get_checkboard(video)

    objp = np.zeros((n_rows * n_cols, 3), np.float32)
    objp[:, :2] = np.mgrid[0:n_rows, 0:n_cols].T.reshape(-1, 2)
    objp = objp * 115
    objpoints = []
    imgpoints = []

    img = cv.imread(f"data/cam{video}/checkboard_axes.jpg")
    window = f"extrinsic_{video}"
    copy = img.copy()

    if auto:
        corners = detect_inner_chessboard_corners(img, video=video)
        corners = np.array(corners, dtype=np.float32).reshape(-1, 1, 2)

        success, rvec, tvec = cv.solvePnP(
            objp, corners, mtx, dist, useExtrinsicGuess=False
        )
    else:
        click_corners(
            img,
            objpoints,
            imgpoints,
            objp,
            window=window,
            n_rows=n_rows,
            n_cols=n_cols,
        )
        success, rvec, tvec = cv.solvePnP(
            objpoints[0], imgpoints[0], mtx, dist, useExtrinsicGuess=False
        )

    if success:
        R, _ = cv.Rodrigues(rvec)

        save_config(mtx, dist, R, tvec, video)

        drawn_img = drawAxes(copy, rvec, tvec, mtx, dist)
        cv.imshow("3D Axes on Checkerboard", drawn_img)
        cv.waitKey(0)
        cv.destroyAllWindows()


def drawAxes(img, rvec, tvec, mtx, dist):

    square = 115
    # coordinate system
    axis = np.float32(
        [[0, 0, 0], [3 * square, 0, 0], [0, 3 * square, 0], [0, 0, -3 * square]]
    )

    # project the axis points and cube points to the 2D image and then convert to pixel coordinates
    imgpts_axis, _ = cv.projectPoints(axis, rvec, tvec, mtx, dist)

    origin = tuple(map(int, imgpts_axis[0].ravel()))
    pt_x = tuple(map(int, imgpts_axis[1].ravel()))
    pt_y = tuple(map(int, imgpts_axis[2].ravel()))
    pt_z = tuple(map(int, imgpts_axis[3].ravel()))
    cv.circle(img, origin, 5, (0, 255, 255), -1)

    cv.arrowedLine(img, origin, pt_x, (0, 0, 255), 2, tipLength=0.2)  # X
    cv.arrowedLine(img, origin, pt_y, (0, 255, 0), 2, tipLength=0.2)  # Y
    cv.arrowedLine(img, origin, pt_z, (255, 0, 0), 2, tipLength=0.2)  # Z
    cv.putText(img, "X", pt_x, cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv.LINE_AA)
    cv.putText(img, "Y", pt_y, cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv.LINE_AA)
    cv.putText(img, "Z", pt_z, cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv.LINE_AA)

    return img

In [ ]:
(
    ret,
    mtx,
    dist,
    rvecs,
    tvecs,
) = calibrate(1)
camera_params(mtx=mtx, dist=dist, video=1, auto=True)

Saved data/cam1/checkboard_axes.jpg


In [17]:
(
    ret,
    mtx,
    dist,
    rvecs,
    tvecs,
) = calibrate(2)
camera_params(mtx=mtx, dist=dist, video=2, auto=True)

Saved data/cam2/checkboard_axes.jpg


In [18]:
(
    ret,
    mtx,
    dist,
    rvecs,
    tvecs,
) = calibrate(3)
camera_params(mtx=mtx, dist=dist, video=3, auto=True)

Saved data/cam3/checkboard_axes.jpg


In [10]:
(
    ret,
    mtx,
    dist,
    rvecs,
    tvecs,
) = calibrate(4)
camera_params(mtx=mtx, dist=dist, video=4, auto=True)

Saved data/cam4/checkboard_axes.jpg
